In [2]:
# import library
import pandas as pd
import numpy as np 
import matlab 
import matplotlib as plt
import os
import numpy as np


In [35]:
# read the file
# mortality
mortality_path_7 = r'data/MortIcd7.csv'
mortality_data_7 = pd.read_csv(mortality_path_7, low_memory=False)
mortality_path_8 = r'data/MortIcd8.csv'
mortality_data_8 = pd.read_csv(mortality_path_8, low_memory=False)
mortality_path_9 = r'data/MortIcd9.csv'
mortality_data_9 = pd.read_csv(mortality_path_9, low_memory=False)

mortality_df = pd.concat([mortality_data_7, mortality_data_8, mortality_data_9], ignore_index=True)

mortality_path_2002 = r'data/Morticd10_part1.csv'
mortality_data_2002 = pd.read_csv(mortality_path_2002, low_memory=False)

mortality_path_2007 = r'data/Morticd10_part2.csv'
mortality_data_2007 = pd.read_csv(mortality_path_2007, low_memory=False)

# population
pop_path = r'data/pop.csv'
pop_data = pd.read_csv(pop_path, low_memory=False)
# country code
country_code_path = r'data/country_codes.csv'
country_code = pd.read_csv(country_code_path, low_memory=False)
# changes to the mortality data file, creating a new data frame with only the number of deaths and the population, country, year, deaths1 - deaths26, pop1 - pop 26, im_death
# creating the country code map 
country_code_map = {}
for index, rows in country_code.iterrows():
    country_code_map[rows['country']] = rows['name']

In [58]:
country_list = []
for key in country_code_map:
    df = mortality_data_2007[mortality_data_2007['Country'] == key]
    year_list = list(df['Year'])
    for i in range (2003, 2008):
        if i not in year_list:
            country_list.append(key)
            break


# delete keys from dictionary
for key in country_list:
    del country_code_map[key]


population_list = []
for i in country_code_map:
    df = pop_data[pop_data['Country'] == key]
    year_list = list(df['Year'])
    for i in range (2003, 2008):
        if i not in year_list:
            population_list.append(key)
            break










[]

In [16]:
# looping through the code map to locate each different countries
# condense the mortality and pop rate 
undesired = ['Admin1', 'SubDiv', 'Sex', 'List', 'Frmat', 'IM_Frmat', 'Cause', 'IM_Deaths1','IM_Deaths2', 'IM_Deaths3', 'IM_Deaths4']
modified_dict = {}
for i in mortality_data_2002.columns.values:
    if i not in undesired:
       modified_dict[i] = []

modified_df = pd.DataFrame(modified_dict)
list_delete_keys = []
consider_population = True

for key in country_code_map: 
    temp_mortality_df = mortality_data_2002[mortality_data_2002['Country'] == key] # filter to the country
    temp_pop_data = pop_data[pop_data['Country'] == key]
    if temp_mortality_df.empty:
        list_delete_keys.append(key)
    else:
        for i in range (1980, 2003):
            temp_temp_mortality_df = temp_mortality_df.loc[temp_mortality_df['Year'] == i] # filter to the year 
            temp_temp_pop_data = temp_pop_data.loc[temp_pop_data['Year'] == i] 
            if temp_temp_mortality_df.empty:
                continue
            else:
                modified_df.loc[modified_df.shape[0]] = ['None'] * modified_df.shape[1]
                modified_df.loc[modified_df.shape[0]-1, ['Country']] = key
                modified_df.loc[modified_df.shape[0]-1, ['Year']] = i
                column_name = modified_df.columns.values 
                k = 1
                for c in column_name[2:]:
                    j = f'Pop{k}'
                    population = temp_temp_pop_data[j].sum()
                    if c in temp_temp_mortality_df.columns.values:
                        if consider_population:
                            if population == 0:
                                modified_df.loc[modified_df.shape[0]-1, [c]] = 0
                            else:
                                modified_df.loc[modified_df.shape[0]-1, [c]] = temp_temp_mortality_df[c].sum() / population
                        else:    
                            modified_df.loc[modified_df.shape[0]-1, [c]] = temp_temp_mortality_df[c].sum() 
                    k += 1
            
                # modified_df.index = modified_df.index + 1
                # modified_df = modified_df.sort_index()      

In [18]:
# delete keys from dictionary
for key in list_delete_keys:
    del country_code_map[key]

In [19]:
# building the mortality model 
country_mortality_model = {}
final_beginning_year = 9999
for key in country_code_map:
    # isolate country
    temp_modified_df = modified_df[modified_df['Country'] == key]
    # temp_modified_df.drop(columns=['Country'], inplace = True) 
    
    # finding alpha 
    last_year = temp_modified_df['Year'].max()
    
    if last_year < final_beginning_year:
        final_beginning_year = last_year
    
    first_year = temp_modified_df['Year'].min()
    duration = last_year - first_year + 1
    last_row = temp_modified_df[temp_modified_df['Year'] == last_year].loc[:, ~temp_modified_df.columns.isin(['Country', 'Year'])].values.astype('float')
    first_row = temp_modified_df[temp_modified_df['Year'] == first_year].loc[:, ~temp_modified_df.columns.isin(['Country', 'Year'])].values.astype('float')
    aphla = (last_row - first_row) / duration

    country_mortality_model[key] = {'a': aphla, 'd': duration}

In [55]:
final_ending_year = 2050
current_year = 2000

mortality_pred_all = []
while current_year <= final_ending_year:
    
    mortality_pred_year = {}
    
    for key in country_code_map: 
        
        print(key)
        
        # isolate the data to the target data row \n",
        temp_modified_df = modified_df[(modified_df['Country'] == key)]  
        # check if there is real data for this year for this country\n",
        if current_year <= 2002:
            df = temp_modified_df[temp_modified_df['Year'] == current_year + 1]
            if not df.empty:
                df = df.loc[:, ~df.columns.isin(['Country', 'Year'])].values.astype('float')
                mortality_pred_year[key] = df 
                continue
 
        # clean the data for the row
        temp_modified_df = temp_modified_df[temp_modified_df['Year'] == current_year].loc[:, ~temp_modified_df.columns.isin(['Country', 'Year'])].values.astype('float')
        # find the aphla, b and k value for the model for future \n",
        aphla = country_mortality_model[key]['a']
        duration = country_mortality_model[key]['d']
        m_hat = (temp_modified_df - aphla)
        s, u, v = np.linalg.svd(m_hat)
        scalar = s[0]
        k = scalar * u[0]
        b = v[0] / scalar 
  
        # compute the next years result",
        change = np.exp(aphla + b * k)
        display(temp_modified_df.shape)
        display(change.shape)
        next_year = temp_mortality_df + change
        mortality_pred_year[key] = next_year
        
        # update parameters
        duration += 1
        aphla = (aphla * duration + change) / duration
        country_mortality_model[key]['a'] = aphla
        country_mortality_model[key]['d'] = duration
    
    current_year += 1
    mortality_pred_all.append(mortality_pred_year)
        
        


1125
1310
1360
1400
1430
2005
2010
2020
2025
2030
2040
2045
2050
2060
2070
2085
2090
2110
2120
2130
2140
2150
2160
2170
2180
2190
2210
2230
2240
2260
2270
2290
2300
2310
2320
2330


(1, 30)

(1, 30)

ValueError: Unable to coerce to DataFrame, shape must be (0, 39): given (1, 30)

In [60]:
key = 1125
temp_modified_df = modified_df[(modified_df['Country'] == key)]  
# clean the data for the row
temp_modified_df = temp_modified_df[temp_modified_df['Year'] == current_year].loc[:, ~temp_modified_df.columns.isin(['Country', 'Year'])].values.astype('float')
# find the aphla, b and k value for the model for future \n",
aphla = country_mortality_model[key]['a']
duration = country_mortality_model[key]['d']
m_hat = (temp_modified_df - aphla)
s, u, v = np.linalg.svd(m_hat)
scalar = s[0]
k = scalar * u[0]
b = v[0] / scalar 

display(aphla)
display(k, b)
change = np.exp(aphla + b * k)
display(temp_modified_df)
display(change)
display(temp_modified_df.shape)
next_year = temp_mortality_df + change
mortality_pred_year[key] = next_year

array([[ 2.49300000e+04, -7.49333333e+02, -3.00000000e+02,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         6.00000000e+01, -1.22666667e+02,  1.76000000e+02,
         1.52666667e+02,  2.39333333e+02, -2.46666667e+01,
         1.27333333e+02,  6.80000000e+02,  5.63333333e+02,
         2.98533333e+03,  3.17133333e+03,  2.30533333e+03,
         1.95666667e+03,  3.72200000e+03,  3.16666667e+03,
         4.19800000e+03,  2.62266667e+03,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00, -7.49333333e+02,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

array([-773286.53820057])

array([ 0.95610872,  0.11802007,  0.03347013, -0.        , -0.        ,
       -0.        ,  0.0123654 ,  0.01235851,  0.01566302,  0.01488883,
        0.01266887,  0.01451036,  0.02079264,  0.02703784,  0.04267586,
        0.0557732 ,  0.05880183,  0.08362834,  0.09795507,  0.11214472,
        0.09051927,  0.06611521,  0.06671955, -0.        , -0.        ,
       -0.        ,  0.11802007, -0.        , -0.        , -0.        ])

array([[764276.,  90514.,  25582.,      0.,      0.,      0.,   9622.,
          9434.,  12288.,  11666.,  10036.,  11196.,  16206.,  21588.,
         33564.,  46114.,  48642.,  66974.,  77704.,  90442.,  73164.,
         55324.,  54216.,      0.,      0.,      0.,  90514.,      0.,
             0.,      0.]])

array([[0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 1., 1., 1.]])

(1, 30)

ValueError: Unable to coerce to DataFrame, shape must be (0, 39): given (1, 30)